In [ ]:
# Block Group Census Population Download for Trimet Residential Analysis

## This script downloads census data exports county level data to an excel file and tract level data to shapefiles.



## TO DO 
- Methodize Scripts
- Further simplify/aggregate countywide stat excel outputs. 
- MHI column is exported as text instead of int. Fix data type


In [ ]:
#import necessary libraries

import requests
import pandas as pd
import geopandas as gpd
import numpy as np
# import geojson
import folium
import os


In [ ]:
# FIPS CODES
Multnomah = "051"
Oregon ="41"

#For each iteration change the county
county = Multnomah
state = Oregon

In [ ]:
# Census Tract Geographies
census_tracts_shp = '/Users/calvindechicago/Documents/GitHub/JWA_2021/cartographic_tracts_oregon/cb_2018_41_tract_500k.shp'
# Verify that filepath is correct
print(os.path.isfile(census_tracts_shp))

# Block Group Geographies
block_groups_shp = "/Users/calvindechicago/Documents/GitHub/JWA_2021/cartographic_tracts_oregon/cb_2018_41_bg_500k/cb_2018_41_bg_500k.shp"

# Set Census Geography level (select all tracts or all block groups) 
# This will be used throughout code to pull either tract or block group data
census_geo = block_groups
#census_geo_string = "block group"
census_geo_string = "tract"

#Set output filepaths
Equity_Tracts = f'/Users/calvindechicago/Documents/GitHub/JWA_2021/code_outputs/equity_tracts_{county}.gpkg'
BG_Pop = f'/Users/calvindechicago/Documents/GitHub/JWA_2021/code_outputs/bg_pop_{county}.gpkg'
EXCEL_FP = f'/Users/calvindechicago/Documents/GitHub/JWA_2021/code_outputs/countylevel_demographics_{county}.xlsx'

In [ ]:
#These are all of the main variables used to build a call url to the the census api website
#Available APIs (--> 2018 ACS Detailed Tables Variables [ html | xml | json ])
#https://www.census.gov/data/developers/data-sets.html

HOST = "https://api.census.gov/data"
year = "2019"
#dataset = "acs/acs5/subject"
dataset = "acs/acs5"
api_key = "f9e79198302081250c07d556f35d8a81cdae528a"
base_url = "/".join([HOST, year, dataset,])

In [ ]:
# Setup request for Table B08006: Sex of Workers by Means of Transportation to Work
# Setup request for Table B08014: SEX OF WORKERS BY VEHICLES AVAILABLE (Total and No vehicle columns only)
# (COUNTY LEVEL)
request_predicates = {}
get_vars_transpo_mode = ["NAME","B08006_001E","B08006_002E","B08006_003E", "B08006_004E",
                         "B08006_008E","B08006_014E","B08006_015E","B08006_016E", 
                         "B08006_017E", "GEO_ID", "B08014_001E", "B08014_002E" 
                        ]
col_names_transpo_mode = ["place_name", "total","total_car_truck_van",
                          "car_truck_van_drove_alone","car_truck_van_carpooled",
                          "public_transportation", "bike", "walk", "taxi_moto_other",
                          "work_from_home","geoid", "total_workers", "no_vehicle",
                          "state_code", "county"
                         ]

request_predicates["key"] = api_key
request_predicates["get"] = ",".join(get_vars_transpo_mode)


#THESE PREDICATES GET AGGREGATE TOTALS FOR ENTIRE COUNTY
request_predicates["for"] = f"county:{county}"  
request_predicates["in"] = f"state:{state}"

transpo_mode_county = requests.get(base_url, params=request_predicates)
num_columns = 15

In [ ]:
#Setting up Means of Transportation to Workdata frame, getting rid of first header row
df_transpo_mode_county = pd.DataFrame(columns=col_names_transpo_mode, data=transpo_mode_county.json()[1:])

In [ ]:
# THESE PREDICATES GET DATA FOR EVERY TRACT OR BLOCK GROUP in County
# These revised predicates replace the 'for' and 'in' predicates above.
#(TRACT LEVEL)

request_predicates["for"] = f"{census_geo_string}:*"
#request_predicates["in"] = "state:06+county:067"
request_predicates["in"] = f"state:{state}+county:{county}"

transpo_mode_tracts = requests.get(base_url, params=request_predicates)



In [ ]:
#Setting up Means of Transportation to Work at Tract Level data frame, getting rid of first header row
#This includes appending 'tract' to columns list
col_names_transpo_mode = ["place_name", "total","total_car_truck_van",
                          "car_truck_van_drove_alone","car_truck_van_carpooled",
                          "public_transportation", "bike", "walk", "taxi_moto_other",
                          "work_from_home","geoid", "total_workers", "no_vehicle","state_code",
                          "county", "tract" 
                         ]


df_transpo_mode_tracts = pd.DataFrame(columns=col_names_transpo_mode, data=transpo_mode_tracts.json()[1:])
num_columns = 16

In [ ]:
dtype_conversion = { "total": int,
                    "total_car_truck_van": int,
                    "car_truck_van_drove_alone": int,
                    "car_truck_van_carpooled": int,
                    "public_transportation": int,
                    "bike": int,
                    "walk": int, 
                    "taxi_moto_other": int, 
                    "work_from_home": int, 
                    "total_workers": int, 
                    "no_vehicle": int
                }

df_transpo_mode_tracts = df_transpo_mode_tracts.astype(dtype_conversion) 

In [ ]:
df_transpo_mode_tracts["pct_no_veh"] = (df_transpo_mode_tracts["no_vehicle"]/df_transpo_mode_tracts["total_workers"]*100)

In [ ]:
#The geoid field in the df_transpo_mode table does not match the Tigerlines geoid field. 
#This slices the the right 11 most digits, which match the geoid codes in the TigerLine file. 
#(... these are state ('06') for California, followed by county, followed by census tract)

df_transpo_mode_tracts.insert(num_columns, "geoid_join",df_transpo_mode_tracts['geoid'].str.slice(-11), True) 



In [ ]:
## SETUP FOR TABLE B08001: SEX BY AGE
#NAME dropped from request because only 50 variables per request are permitted.
request_predicates = {}
get_vars_age = ["B01001_001E", "B01001_002E","B01001_003E", "B01001_004E","B01001_005E", 
                "B01001_006E", "B01001_007E","B01001_008E", "B01001_009E","B01001_010E", 
                "B01001_011E", "B01001_012E","B01001_013E", "B01001_014E","B01001_015E", 
                "B01001_016E", "B01001_017E","B01001_018E", "B01001_019E","B01001_020E", 
                "B01001_021E", "B01001_022E","B01001_023E", "B01001_024E","B01001_025E", 
                "B01001_026E", "B01001_027E","B01001_028E", "B01001_029E","B01001_030E", 
                "B01001_031E", "B01001_032E","B01001_033E", "B01001_034E","B01001_035E", 
                "B01001_036E", "B01001_037E","B01001_038E", "B01001_039E","B01001_040E", 
                "B01001_041E", "B01001_042E","B01001_043E", "B01001_044E","B01001_045E", 
                "B01001_046E", "B01001_047E","B01001_048E", "B01001_049E","GEO_ID"
                ]

col_names_age = ["total_pop","total_male","tl_m0_5","tl_m5_9", "tl_m10_14", "tl_m15_17", 
                 "tl_m18_19", "tl_m20", "tl_m21", "tl_m22_24", "tl_m25_29", "tl_m30_34",
                 "tl_m35_39", "tl_m40_44", "tl_m45_49", "tl_m50_54", "tl_m55_59", "tl_m60_61",
                 "tl_m62_64", "tl_m65_66", "tl_m67_69", "tl_m70_74", "tl_m75_79", "tl_m80_84",
                 "tl_m85_pl","total_female", "tl_f0_5","tl_f5_9", "tl_f10_14", "tl_f15_17", 
                 "tl_f18_19", "tl_f20", "tl_f21", "tl_f22_24", "tl_f25_29", "tl_f30_34",
                 "tl_f35_39", "tl_f40_44", "tl_f45_49", "tl_f50_54", "tl_f55_59", "tl_f60_61",
                 "tl_f62_64", "tl_f65_66", "tl_f67_69", "tl_f70_74", "tl_f75_79", "tl_f80_84",
                 "tl_f85_pl","geoid","state","county"
                ]

request_predicates["key"] = api_key
request_predicates["get"] = ",".join(get_vars_age)

#THESE PREDICATES GET AGGREGATE TOTALS FOR ENTIRE COUNTY
request_predicates["for"] = f"county:{county}"
request_predicates["in"] = f"state:{state}"


age_county = requests.get(base_url, params=request_predicates)
num_columns = 49

In [ ]:
#Setting up AGE data frame, getting rid of first header row
df_age_county = pd.DataFrame(columns=col_names_age, data=age_county.json()[1:])

In [ ]:
# THESE PREDICATES GET DATA FOR EVERY TRACT

request_predicates["for"] = f"{census_geo_string}:*"
request_predicates["in"] = f"state:{state}+county:{county}"

age_tracts = requests.get(base_url, params=request_predicates)
num_columns = 50

In [ ]:
#Setting up AGE TRACTS data frame, getting rid of first header row
col_names_age = ["total_pop","total_male","tl_m0_5","tl_m5_9", "tl_m10_14", "tl_m15_17", 
                 "tl_m18_19", "tl_m20", "tl_m21", "tl_m22_24", "tl_m25_29", "tl_m30_34",
                 "tl_m35_39", "tl_m40_44", "tl_m45_49", "tl_m50_54", "tl_m55_59", "tl_m60_61",
                 "tl_m62_64", "tl_m65_66", "tl_m67_69", "tl_m70_74", "tl_m75_79", "tl_m80_84",
                 "tl_m85_pl","total_female", "tl_f0_5","tl_f5_9", "tl_f10_14", "tl_f15_17", 
                 "tl_f18_19", "tl_f20", "tl_f21", "tl_f22_24", "tl_f25_29", "tl_f30_34",
                 "tl_f35_39", "tl_f40_44", "tl_f45_49", "tl_f50_54", "tl_f55_59", "tl_f60_61",
                 "tl_f62_64", "tl_f65_66", "tl_f67_69", "tl_f70_74", "tl_f75_79", "tl_f80_84",
                 "tl_f85_pl","geoid","state","county", 'tract'
                ]
df_age_tracts = pd.DataFrame(columns=col_names_age, data=age_tracts.json()[1:])

In [ ]:
# Use dictionary to convert specific columns 

dtype_conversion = { "total_pop": int,
                    "total_male": int,
                    "tl_m0_5": int,
                    "tl_m5_9": int,
                    "tl_m10_14": int,
                    "tl_m15_17": int,
                    "tl_m18_19": int, 
                    "tl_m20": int, 
                    "tl_m21": int, 
                    "tl_m22_24": int, 
                    "tl_m25_29": int, 
                    "tl_m30_34": int,
                    "tl_m35_39": int, 
                    "tl_m40_44": int, 
                    "tl_m45_49": int, 
                    "tl_m50_54": int, 
                    "tl_m55_59": int, 
                    "tl_m60_61": int,
                    "tl_m62_64": int, 
                    "tl_m65_66": int, 
                    "tl_m67_69": int, 
                    "tl_m70_74": int, 
                    "tl_m75_79": int, 
                    "tl_m80_84": int,
                    "tl_m85_pl": int,
                    "total_female": int, 
                    "tl_f0_5": int,
                    "tl_f5_9": int,
                    "tl_f10_14": int,
                    "tl_f15_17": int,
                    "tl_f18_19": int,
                    "tl_f20": int,
                    "tl_f21": int,
                    "tl_f22_24": int,
                    "tl_f25_29": int,
                    "tl_f30_34": int,
                    "tl_f35_39": int,
                    "tl_f40_44": int,
                    "tl_f45_49": int,
                    "tl_f50_54": int,
                    "tl_f55_59": int,
                    "tl_f60_61": int,
                    "tl_f62_64": int,
                    "tl_f65_66": int,
                    "tl_f67_69": int,
                    "tl_f70_74": int,
                    "tl_f75_79": int,
                    "tl_f80_84": int,
                    "tl_f85_pl": int,
                    "county": int,
                    'tract': int
                }
df_age_tracts = df_age_tracts.astype(dtype_conversion) 

In [ ]:
age_18_under = df_age_tracts["tl_m0_5"] + df_age_tracts["tl_f0_5"] + df_age_tracts["tl_m5_9"] + df_age_tracts["tl_m10_14"] + df_age_tracts["tl_m15_17"] + df_age_tracts["tl_f5_9"] + df_age_tracts["tl_f10_14"] + df_age_tracts["tl_f15_17"]

age_65_over = df_age_tracts["tl_f65_66"] + df_age_tracts["tl_f67_69"] + df_age_tracts["tl_f70_74"] + df_age_tracts["tl_f75_79"] + df_age_tracts["tl_f80_84"] + df_age_tracts["tl_f85_pl"] + df_age_tracts["tl_m65_66"] + df_age_tracts["tl_m67_69"] + df_age_tracts["tl_m70_74"] + df_age_tracts["tl_m75_79"] + df_age_tracts["tl_m80_84"] + df_age_tracts["tl_m85_pl"]

age_vulnerable = age_18_under + age_65_over



In [ ]:
df_age_tracts["age_18_under"] = age_18_under 
df_age_tracts["age_65_over"] = age_65_over
df_age_tracts["age_vulnerable"] = age_vulnerable

df_age_tracts["pct_age_18_under"] = df_age_tracts["age_18_under"]/df_age_tracts["total_pop"]*100
df_age_tracts["pct_age_65_over"] = df_age_tracts["age_65_over"]/df_age_tracts["total_pop"]*100
df_age_tracts["pct_age_vulnerable"] = df_age_tracts["age_vulnerable"]/df_age_tracts["total_pop"]*100




In [ ]:
df_age_tracts_simple = df_age_tracts[["total_pop","age_18_under","age_65_over","age_vulnerable", "pct_age_18_under","pct_age_65_over","pct_age_vulnerable","geoid","state","county","tract"]]

In [ ]:
#Add geoid_join column 
df_age_tracts_simple.insert(8, "geoid_join",df_transpo_mode_tracts['geoid'].str.slice(-11), True) 


In [ ]:
#B19013 Median Household Income - TRACT

# THESE PREDICATES GET DATA FOR EVERY TRACT
#NAME dropped from request because only 50 variables per request are permitted.
request_predicates = {}
get_vars_mhi = ["NAME","B19013_001E", "GEO_ID"
                ]

col_names_mhi = ["place_name", "mhi",
                 "geoid","state","county", "tract"
                ]

request_predicates["key"] = api_key
request_predicates["get"] = ",".join(get_vars_mhi)



request_predicates["for"] = f"{census_geo_string}:*"
request_predicates["in"] = f"state:{state}+county:{county}"

mhi_tracts = requests.get(base_url, params=request_predicates)
num_columns = 50



In [ ]:
#B19013 Median Household Income - COUNTY

request_predicates = {}
get_vars_mhi_county = ["NAME","B19013_001E", "GEO_ID"
                ]

col_names_mhi_county = ["place_name", "mhi",
                 "geoid","state","county"
                ]

request_predicates["key"] = api_key
request_predicates["get"] = ",".join(get_vars_mhi_county)


#THESE PREDICATES GET AGGREGATE TOTALS FOR ENTIRE COUNTY (SACRAMENTO)
#request_predicates["for"] = "county:067" #Sacramento
request_predicates["for"] = f"county:{county}"  
request_predicates["in"] = f"state:{state}"

mhi_county = requests.get(base_url, params=request_predicates)
num_columns = 15

In [ ]:
df_mhi_county = pd.DataFrame(columns=col_names_mhi_county, data=mhi_county.json()[1:])

In [ ]:
#Setting up AGE data frame, getting rid of first header row
df_mhi = pd.DataFrame(columns=col_names_mhi, data=mhi_tracts.json()[1:])

In [ ]:
df_mhi.insert(6, "geoid_join",df_mhi['geoid'].str.slice(-11), True) 


In [ ]:

df_age_simplified = df_age_tracts_simple[['total_pop', 'age_18_under', 'age_65_over', 'age_vulnerable',
       'pct_age_18_under', 'pct_age_65_over', 'pct_age_vulnerable',
       'geoid_join']]


In [ ]:

#df_transpo_mode_tracts
df_mhi_simplified = df_mhi[['mhi','geoid_join']]


In [ ]:
#Convert MHI value to numeric type
df_mhi_simplified.loc[:,"mhi"] = df_mhi_simplified.mhi.apply(pd.to_numeric)

In [ ]:
# Merge equity Tracts
transpo_mhi = pd.merge(df_transpo_mode_tracts, df_mhi_simplified, on="geoid_join")





In [ ]:
transpo_mhi_age = pd.merge(transpo_mhi,df_age_simplified,  on= "geoid_join" )


In [ ]:
# Join the census data to Tigerline Cartographic Boundary census tract geometries. 
# Census Tract Tigerline California Census Tracts location (ftp url included in notes below)



#This reads the census tracts shapefile into a geodataframe
gdf = gpd.read_file(census_tracts_shp)

#Make tigerline boundary columns lowercase
gdf.columns = map(str.lower, gdf.columns)


In [ ]:
# Merge tract geographies data with the data produced in earlier steps

final_equity_geom = gdf.merge(transpo_mhi_age,left_on='geoid',right_on='geoid_join')


In [ ]:
# Export to geopackage
# countries_gdf.to_file("package.gpkg", layer='countries', driver="GPKG")

final_equity_geom.to_file(Equity_Tracts, layer='equity_tracts', driver="GPKG")

In [ ]:
#Export county level dataframes to excel

with pd.ExcelWriter(EXCEL_FP) as writer:
    df_age_county.to_excel(writer, sheet_name='county_age')
    df_transpo_mode_county.to_excel(writer, sheet_name='county_transpo_mode')
    df_mhi_county.to_excel(writer, sheet_name='county_mhi')

# OTHER EXAMPLES FOUND WHILE TROUBLESHOOTING
# with pd.ExcelWriter("test.xlsx", engine='openpyxl', mode='a') as writer:
#     df.to_excel(writer)
# with pd.ExcelWriter(Excel_FP) as writer:
#     bike_crashes_by_year.to_excel(writer, sheet_name='b_crashes_by_year')

In [ ]:
## SETUP FOR TABLE B01003: Total Population
# Create Block Group Populations
# TABLE: B01003_001E

HOST = "https://api.census.gov/data"
year = "2015"
dataset = "acs/acs5"
api_key = "f9e79198302081250c07d556f35d8a81cdae528a"
base_url = "/".join([HOST, year, dataset,])

In [ ]:

# Only 50 variables per request are permitted.

request_predicates = {}
get_vars_age = ["NAME", "B01003_001E","GEO_ID"]

col_bg_pop = ["name_string","total_pop","geoid","state", "county", "tract", "block_group"]

request_predicates["key"] = api_key
request_predicates["get"] = ",".join(get_vars_age)

#THESE PREDICATES GET DATA FOR EVERY BLOCK GROUP IN COUNTY

request_predicates["for"] = f"block group:*"
request_predicates["in"] = f"state:{state}+county:{county}"


bg_pop = requests.get(base_url, params=request_predicates)


In [ ]:


#Setting up Block Group Population data frame, getting rid of first header row
df_bg_pop = pd.DataFrame(columns=col_bg_pop, data=bg_pop.json()[1:])




In [ ]:
#The geoid field in the df_transpo_mode table does not match the Tigerlines geoid field. 
#This slices the the right 11 most digits, which match the geoid codes in the TigerLine file. 
#(... these are state ('06') for California, followed by county, followed by census tract)
num_columns = df_bg_pop.shape[1]

df_bg_pop.insert(num_columns, "geoid_join", df_bg_pop['geoid'].str.slice(start=9), True) 


In [ ]:
df_bg_pop

In [ ]:
df_bg_pop.shape

In [ ]:
df_bg_pop[df_bg_pop['geoid_join']==410510072023]

In [ ]:
# Join the census data to Tigerline Cartographic Boundary block group geometries. 
# Census Tract Tigerline California block group location (ftp url included in notes below)


#This reads the census tracts shapefile into a geodataframe
bg_gdf = gpd.read_file(block_groups_shp)

#Make tigerline boundary columns lowercase
bg_gdf.columns = map(str.lower, bg_gdf.columns)

In [ ]:
bg_gdf = bg_gdf[bg_gdf['countyfp']=='051']


In [ ]:
bg_gdf

In [ ]:
bg_gdf.sort_values(by=['geoid'],ascending=True)

In [ ]:
# Merge block group geographies data with the population data produced in earlier steps

final_bg_pop = bg_gdf.merge(df_bg_pop,left_on='affgeoid',right_on='geoid')

In [ ]:
final_bg_pop.shape

In [ ]:
final_bg_pop.plot()

In [ ]:
final_bg_pop.to_file(BG_Pop, layer='BG_Pop', driver="GPKG")